In [ ]:
#보급품목 텍스트 크롤링 최종

import re
import requests
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
from html.parser import HTMLParser


#컬럼별 리스트
product_name_list=[] #제품명
contact_number_list=[] #연락처
manufacturer_list=[] #제조원/공급원
product_summary_list=[] #제품개요
product_specifications_list=[] #제품사양
main_functions_list=[] # 주요기능
product_features_list=[] #제품특징
main_items_list=[] #구성품목
product_user_list=[] #사용가능 대상
product_effects_list=[] #기대효과
how_to_use_list=[] #사용방법


#페이지별 링크 주소 불러오기
for i in range(1,22):
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱
    url=soup.find_all('td',{'class':'align_left'})
    #print(url)
    
    
    #제품별 정보에 관한 링크 주소 가져오기
    main_url='https://www.at4u.or.kr'
    sub_url = [main_url + tag.find('a').get('href','') if tag.find('a') else '' for tag in url]
    #print(sub_url)

    #태그로 불러온 데이터에서 텍스트만 추출
    def to_text(data): 
        rows=[]
        for tag in data:
            name=tag.get_text()
            rows.append(name)
            
        return rows

    #제품별 정보 불러오기
    for i in sub_url:
        if i !='':
            response1=requests.get(i) #제품별 주소 불러오기
            soup=BeautifulSoup(response1.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱
            
            
            #제품명, 연락처, 제조/공급원 
            title_tag=soup.select('div>dl>dt') #제품명, 연락처, 제조/공급원 태그 불러오기
            #print(title_tag)
            title=to_text(title_tag) #텍스트로 변환
            #print(title)

            if '제품명' in title: #제품명 항목 유무 확인
                product_name_tag=soup.find_all('dd','marb_5 padt_2') #제품명 태그 불러오기
                product_name=to_text(product_name_tag)[0] #제품명만 추출
                product_name_list.append(product_name)
        
            else:
                product_name_list.append('none')
                

            if '연락처' in title: #연락처 항목 유무 확인
                contact_number=to_text(product_name_tag)[1] #연락처만 추출
                contact_number_list.append(contact_number)
                
            else:
                contact_number_list.append('none')
                
            if '제조,공급원' in title: #제조,공급원 항목 유무 확인
                manufacturer_tag=soup.find_all('dd','marb_6 padt_2') #제조원/공급원 태그 불러오기
                #print(manufacturer_tag)
                manufacturer=to_text(manufacturer_tag)[0] #제조원/공급원 추출
                manufacturer_list.append(manufacturer)
                
            else:
                manufacturer_list.append('none')


            #제품개요,제품사양, 주요기능, 제품특징, 구성품목, 사용가능 대상, 기대효과, 사용방법
            explan_tag=soup.find_all('dd',{'class':'padb_10'}) #설명 태그 불러오기
            explan=to_text(explan_tag) #설명 텍스트만 추출
            #print(explan)

            for i in range(len(explan)):
                a=re.sub('\t|\xa0','',explan[i]) #\t, \xa0 등 제거
                explan[i]=a

            bold_original=['제품개요','제품사양','주요기능','제품특징','구성품목','사용가능 대상','기대효과','사용방법'] #원래 항목들
            bold=soup.select('div>dl>dt>img') #존재하는 항목 불러오기
            bold_list=[]

            for i in range(len(bold)):
                bold_list.append(bold[i]['alt'])
            bold_list=bold_list[1:] #존재하는 항목 확인
            #print(bold_list)

            j=0
            result=[]
            for i in range(len(bold_original)):
                if bold_original[i] in bold_list: #원래 항목이 현 페이지의 항목에 존재하면 값 추가
                    result.append(explan[j])
                    j+=1
                else:
                    result.append('none') #항목 존재하지 않으면 'none' 입력
            #print(result)

            product_summary_list.append(result[0]) #제품개요
            product_specifications_list.append(result[1]) #제품사양
            main_functions_list.append(result[2]) # 주요기능
            product_features_list.append(result[3]) #제품특징
            main_items_list.append(result[4]) #구성품목
            product_user_list.append(result[5]) #사용가능 대상
            product_effects_list.append(result[6]) #기대효과
            how_to_use_list.append(result[7]) #사용방법
         
            
import pandas as pd
df=pd.DataFrame(list(zip(product_name_list,contact_number_list,manufacturer_list,product_summary_list,product_specifications_list, main_functions_list,
                        product_features_list,main_items_list,product_user_list, product_effects_list, how_to_use_list)),
                columns=['제품명','연락처','제조원/공급원','제품개요','제품사양','주요기능','제품특징','구성품목','사용자','기대효과','사용방법'])


print(df)

#df.to_excel('product2.xlsx') #엑셀파일로 만들기

이미지 크롤링해서 데이터 프레임 만들고, 원래 있던 데이터 프레임이랑 합쳐서 엑셀파일 만들기!

In [ ]:
import re
import requests
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
from html.parser import HTMLParser

#페이지별 링크 주소 불러오기
for i in range(1,22):
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱
    url=soup.find_all('td',{'class':'align_left'})
    #print(url)
    
    
    #제품별 정보에 관한 링크 주소 가져오기
    main_url='https://www.at4u.or.kr'
    sub_url = [main_url + tag.find('a').get('href','') if tag.find('a') else '' for tag in url]
    #print(sub_url)

    #태그로 불러온 데이터에서 텍스트만 추출
    def to_text(data): 
        rows=[]
        for tag in data:
            name=tag.get_text()
            rows.append(name)
            
        return rows

    #제품별 정보 불러오기
    for i in sub_url:
        if i !='':
            response1=requests.get(i) #제품별 주소 불러오기
            soup=BeautifulSoup(response1.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱
            
            
            #제품명, 연락처, 제조/공급원 
            title_tag=soup.select('div>dl>dt') #제품명, 연락처, 제조/공급원 태그 불러오기
            #print(title_tag)
            title=to_text(title_tag) #텍스트로 변환
            #print(title)



In [61]:
#이미지 불러오기

import re
import requests
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
from html.parser import HTMLParser


response1=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage=1')
soup=BeautifulSoup(response1.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱

main_url='https://www.at4u.or.kr'
url=soup.find_all('td',{'class':'align_center'})
sub_url = [main_url + tag.find('img').get('src','') if tag.find('img') else '' for tag in url]
#print(url)
print(sub_url)
sub_url = list(filter(None,sub_url))
print(sub_url)


#image_tag=soup.find_all('td',{'class':'align_left'})
#image_url=image_tag.find('img')['src']
#print(image_tag)
#print(image_url)
#image_url=main_url+image_url
#print(image_url)


['', 'https://www.at4u.or.kr/FILES/PDT/040704140422멀티모니터24_1.bmp', '', '', 'https://www.at4u.or.kr/FILES/PDT/04070419핑키밍키.png', '', '', 'https://www.at4u.or.kr/FILES/PDT/0407041927_2.png', '', '', 'https://www.at4u.or.kr/FILES/PDT/040724인치_화면확대터치모니터.jpg', '', '', 'https://www.at4u.or.kr/FILES/PDT/04070414042227인치모니터_1.bmp', '', '', 'https://www.at4u.or.kr/FILES/PDT/27인치_멀티확대_터치모니터.png', '']
['https://www.at4u.or.kr/FILES/PDT/040704140422멀티모니터24_1.bmp', 'https://www.at4u.or.kr/FILES/PDT/04070419핑키밍키.png', 'https://www.at4u.or.kr/FILES/PDT/0407041927_2.png', 'https://www.at4u.or.kr/FILES/PDT/040724인치_화면확대터치모니터.jpg', 'https://www.at4u.or.kr/FILES/PDT/04070414042227인치모니터_1.bmp', 'https://www.at4u.or.kr/FILES/PDT/27인치_멀티확대_터치모니터.png']


In [13]:
#제품 정보 페이지에서 이미지 가져오기


image_list=[]

count=0

#페이지별 링크 주소 불러오기
for i in range(1,5):
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱
    url=soup.find_all('td',{'class':'align_left'})
    #print(url)
    
    #제품별 정보에 관한 링크 주소 가져오기
    main_url='https://www.at4u.or.kr'
    sub_url = [main_url + tag.find('a').get('href','') if tag.find('a') else '' for tag in url]
    #print(sub_url)

    #제품별 정보 불러오기
    for i in sub_url:
        if i !='':
            response1=requests.get(i) #제품별 주소 불러오기
            soup=BeautifulSoup(response1.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱
            image_tag=soup.find('div',{'class':'img_big'}) #이미지 태그 불러오기
            image_url=image_tag.find('img')['src'] #이미지 링크 불러오기
            #print(image_tag)
            #print(image_url)
            image_url=main_url+image_url 
            count+=1
            print(image_url)
            
print(count)            
            

https://www.at4u.or.kr/FILES/PDT/040704140422모니터_모양캡처.JPG
https://www.at4u.or.kr/IMG/noneimage.gif
https://www.at4u.or.kr/FILES/PDT/0407041927_3.png
https://www.at4u.or.kr/FILES/PDT/040724인치_화면확대터치모니터_1.jpg
https://www.at4u.or.kr/FILES/PDT/04070414042204250426041827인치멀티터치모니터_앞_1.jpg
https://www.at4u.or.kr/IMG/noneimage.gif
https://www.at4u.or.kr/FILES/PDT/0410041927_1.png
https://www.at4u.or.kr/FILES/PDT/040727인치_화면확대터치모니터_1.jpg
https://www.at4u.or.kr/FILES/PDT/04070413042104233스타2.bmp
https://www.at4u.or.kr/IMG/noneimage.gif
https://www.at4u.or.kr/IMG/noneimage.gif
https://www.at4u.or.kr/IMG/noneimage.gif
https://www.at4u.or.kr/FILES/PDT/0407IMG_2759_1.JPG
https://www.at4u.or.kr/FILES/PDT/0407강한손_터치모니터_32_1.png
https://www.at4u.or.kr/FILES/PDT/04070419042704250426골전도_1.png
https://www.at4u.or.kr/FILES/PDT/0407네비게이션신호기세트_1.png
https://www.at4u.or.kr/FILES/PDT/040711._노바캠리더_1.png
https://www.at4u.or.kr/IMG/noneimage.gif
https://www.at4u.or.kr/FILES/PDT/04070409다빈치프로.png
https://www.at4u

이미지 링크 불러오기 최종

In [68]:
main_url='https://www.at4u.or.kr' #메인 링크
image_url_list=[] #이미지 링크 주소 리스트

#count=0

for i in range(1,22): #페이지별 링크 주소 불러오기
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱

    url=soup.find_all('td',{'class':'align_center'})
    sub_url = [main_url + tag.find('img').get('src','') if tag.find('img') else '' for tag in url] #이미지 링크 불러오기
    #print(url)
    #print(sub_url)
    sub_url = list(filter(None,sub_url)) #이미지 링크 리스트에서 불필요한 공백 제거

    for i in range(len(sub_url)):
        image_url_list.append(sub_url[i]) #이미지 링크 리스트에 하나씩 추가
        #count+=1

print(image_url_list)
#print(count)

['https://www.at4u.or.kr/FILES/PDT/040704140422멀티모니터24_1.bmp', 'https://www.at4u.or.kr/FILES/PDT/04070419핑키밍키.png', 'https://www.at4u.or.kr/FILES/PDT/0407041927_2.png', 'https://www.at4u.or.kr/FILES/PDT/040724인치_화면확대터치모니터.jpg', 'https://www.at4u.or.kr/FILES/PDT/04070414042227인치모니터_1.bmp', 'https://www.at4u.or.kr/FILES/PDT/27인치_멀티확대_터치모니터.png', 'https://www.at4u.or.kr/FILES/PDT/0410041927.png', 'https://www.at4u.or.kr/FILES/PDT/040727인치_화면확대터치모니터.jpg', 'https://www.at4u.or.kr/FILES/PDT/04070413042104233스타.bmp', 'https://www.at4u.or.kr/FILES/PDT/강한손_보이고1.png', 'https://www.at4u.or.kr/FILES/PDT/강한손_안마사랑1.png', 'https://www.at4u.or.kr/FILES/PDT/강한손_이지스_울트라1.png', 'https://www.at4u.or.kr/FILES/PDT/0407IMG_2759.png', 'https://www.at4u.or.kr/FILES/PDT/0407강한손_터치모니터_32.png', 'https://www.at4u.or.kr/FILES/PDT/04070419042704250426골전도.png', 'https://www.at4u.or.kr/FILES/PDT/0407네비게이션신호기세트.png', 'https://www.at4u.or.kr/FILES/PDT/040711._노바캠리더.png', 'https://www.at4u.or.kr/FILES/PDT/04070413뉴포트.png

#### 텍스트 데이터랑 이미지 데이터 합치기

In [75]:
#이미지 데이터 데이터 프레임 만들기

main_url='https://www.at4u.or.kr' #메인 링크
image_url_list=[] #이미지 링크 주소 리스트
num=[]

#count=0

for i in range(1,22): #페이지별 링크 주소 불러오기
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱

    url=soup.find_all('td',{'class':'align_center'})
    sub_url = [main_url + tag.find('img').get('src','') if tag.find('img') else '' for tag in url] #이미지 링크 불러오기
    #print(url)
    #print(sub_url)
    sub_url = list(filter(None,sub_url)) #이미지 링크 리스트에서 불필요한 공백 제거

    for i in range(len(sub_url)):
        image_url_list.append(sub_url[i]) #이미지 링크 리스트에 하나씩 추가
        #count+=1

for i in range(1,122): #제품번호 추가
    num.append(i)

#print(image_url_list)
df_image=pd.DataFrame(list(zip(num,image_url_list)),
                columns=['제품번호','제품이미지'])

print(df_image)

     제품번호                                              제품이미지
0       1  https://www.at4u.or.kr/FILES/PDT/040704140422멀...
1       2  https://www.at4u.or.kr/FILES/PDT/04070419핑키밍키.png
2       3  https://www.at4u.or.kr/FILES/PDT/0407041927_2.png
3       4  https://www.at4u.or.kr/FILES/PDT/040724인치_화면확대...
4       5  https://www.at4u.or.kr/FILES/PDT/0407041404222...
..    ...                                                ...
116   117  https://www.at4u.or.kr/FILES/PDT/040904200421한...
117   118  https://www.at4u.or.kr/FILES/PDT/040904130422헤...
118   119  https://www.at4u.or.kr/FILES/PDT/040904190427히...
119   120   https://www.at4u.or.kr/FILES/PDT/0409쿼티케이스모델.jpg
120   121    https://www.at4u.or.kr/FILES/PDT/쿼티케이스에스이_1.png

[121 rows x 2 columns]


In [77]:
#보급품목 텍스트 크롤링+인덱스 포함 데이터 프레임 만들기

import re
import requests
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
from html.parser import HTMLParser


#컬럼별 리스트
num=[] #인덱스값
product_name_list=[] #제품명
contact_number_list=[] #연락처
manufacturer_list=[] #제조원/공급원
product_summary_list=[] #제품개요
product_specifications_list=[] #제품사양
main_functions_list=[] # 주요기능
product_features_list=[] #제품특징
main_items_list=[] #구성품목
product_user_list=[] #사용가능 대상
product_effects_list=[] #기대효과
how_to_use_list=[] #사용방법


#페이지별 링크 주소 불러오기

for i in range(1,22):
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱
    url=soup.find_all('td',{'class':'align_left'})
    #print(url)
    
    
    #제품별 정보에 관한 링크 주소 가져오기
    main_url='https://www.at4u.or.kr'
    sub_url = [main_url + tag.find('a').get('href','') if tag.find('a') else '' for tag in url]
    #print(sub_url)

    #태그로 불러온 데이터에서 텍스트만 추출
    def to_text(data): 
        rows=[]
        for tag in data:
            name=tag.get_text()
            rows.append(name)
            
        return rows

    #제품별 정보 불러오기
    for i in sub_url:
        if i !='':
            response1=requests.get(i) #제품별 주소 불러오기
            soup=BeautifulSoup(response1.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱
            
            
            #제품명, 연락처, 제조/공급원 
            title_tag=soup.select('div>dl>dt') #제품명, 연락처, 제조/공급원 태그 불러오기
            #print(title_tag)
            title=to_text(title_tag) #텍스트로 변환
            #print(title)

            if '제품명' in title: #제품명 항목 유무 확인
                product_name_tag=soup.find_all('dd','marb_5 padt_2') #제품명 태그 불러오기
                product_name=to_text(product_name_tag)[0] #제품명만 추출
                product_name_list.append(product_name)
        
            else:
                product_name_list.append('none')
                

            if '연락처' in title: #연락처 항목 유무 확인
                contact_number=to_text(product_name_tag)[1] #연락처만 추출
                contact_number_list.append(contact_number)
                
            else:
                contact_number_list.append('none')
                
            if '제조,공급원' in title: #제조,공급원 항목 유무 확인
                manufacturer_tag=soup.find_all('dd','marb_6 padt_2') #제조원/공급원 태그 불러오기
                #print(manufacturer_tag)
                manufacturer=to_text(manufacturer_tag)[0] #제조원/공급원 추출
                manufacturer_list.append(manufacturer)
                
            else:
                manufacturer_list.append('none')


            #제품개요,제품사양, 주요기능, 제품특징, 구성품목, 사용가능 대상, 기대효과, 사용방법
            explan_tag=soup.find_all('dd',{'class':'padb_10'}) #설명 태그 불러오기
            explan=to_text(explan_tag) #설명 텍스트만 추출
            #print(explan)

            for i in range(len(explan)):
                a=re.sub('\t|\xa0','',explan[i]) #\t, \xa0 등 제거
                explan[i]=a

            bold_original=['제품개요','제품사양','주요기능','제품특징','구성품목','사용가능 대상','기대효과','사용방법'] #원래 항목들
            bold=soup.select('div>dl>dt>img') #존재하는 항목 불러오기
            bold_list=[]

            for i in range(len(bold)):
                bold_list.append(bold[i]['alt'])
            bold_list=bold_list[1:] #존재하는 항목 확인
            #print(bold_list)

            j=0
            result=[]
            for i in range(len(bold_original)):
                if bold_original[i] in bold_list: #원래 항목이 현 페이지의 항목에 존재하면 값 추가
                    result.append(explan[j])
                    j+=1
                else:
                    result.append('none') #항목 존재하지 않으면 'none' 입력
            #print(result)

            product_summary_list.append(result[0]) #제품개요
            product_specifications_list.append(result[1]) #제품사양
            main_functions_list.append(result[2]) # 주요기능
            product_features_list.append(result[3]) #제품특징
            main_items_list.append(result[4]) #구성품목
            product_user_list.append(result[5]) #사용가능 대상
            product_effects_list.append(result[6]) #기대효과
            how_to_use_list.append(result[7]) #사용방법
            

for i in range(1,122): #제품번호 추가
    num.append(i)
         
            
import pandas as pd
df_text=pd.DataFrame(list(zip(num,product_name_list, contact_number_list,manufacturer_list,product_summary_list,product_specifications_list, main_functions_list,
                        product_features_list,main_items_list,product_user_list, product_effects_list, how_to_use_list)),
                columns=['제품번호','제품명','연락처','제조원/공급원','제품개요','제품사양','주요기능','제품특징','구성품목','사용자','기대효과','사용방법'])


print(df_text)

#df_text.to_excel('웹크롤링 텍스트 인덱스 포함.xlsx') #엑셀파일로 만들기

     제품번호               제품명            연락처   
0       1   24인치 멀티터치모니터/TV   02-2277-3296  \
1       2   24인치 멀티확대 터치모니터  070-4481-4887   
2       3  24인치터치모니터 LED TV  070-4481-4887   
3       4     24인치화면확대터치모니터   02-2277-3296   
4       5      27인치 멀티터치모니터   02-2277-3296   
..    ...               ...            ...   
116   117          한소네U2 쿼티   042-879-3000   
117   118         헤드마우스나노세트  010-8258-7807   
118   119             히어링스톤   042-716-1100   
119   120  Qwertycase Patch   031-443-4882   
120   121     Qwertycase SE   031-443-4882   

                                               제조원/공급원   
0    제조원 : 한스정보/대한민국공급원 : (재)파라다이스복지재단/대한민국www.para...  \
1                                              유토/대한민국   
2       제조원 : 유토/대한민국공급원 : (주)핑키밍키/대한민국www.goodedu.com   
3                                       (주)한스정보 / 대한민국   
4    제조원 : 한스정보/대한민국공급원 : (재)파라다이스복지재단/대한민국www.para...   
..                                                 ...   
116                                    ㈜셀

In [79]:
#'제품번호' 컬럼에 따라 두 데이터 프레임(이미지, 텍스트) 합치기

df_final=pd.merge(df_image,df_text)
print(df_final)
#df_final.to_excel('웹크롤링 텍스트,이미지,인덱스 포함.xlsx') #엑셀파일로 만들기


     제품번호                                              제품이미지   
0       1  https://www.at4u.or.kr/FILES/PDT/040704140422멀...  \
1       2  https://www.at4u.or.kr/FILES/PDT/04070419핑키밍키.png   
2       3  https://www.at4u.or.kr/FILES/PDT/0407041927_2.png   
3       4  https://www.at4u.or.kr/FILES/PDT/040724인치_화면확대...   
4       5  https://www.at4u.or.kr/FILES/PDT/0407041404222...   
..    ...                                                ...   
116   117  https://www.at4u.or.kr/FILES/PDT/040904200421한...   
117   118  https://www.at4u.or.kr/FILES/PDT/040904130422헤...   
118   119  https://www.at4u.or.kr/FILES/PDT/040904190427히...   
119   120   https://www.at4u.or.kr/FILES/PDT/0409쿼티케이스모델.jpg   
120   121    https://www.at4u.or.kr/FILES/PDT/쿼티케이스에스이_1.png   

                  제품명            연락처   
0     24인치 멀티터치모니터/TV   02-2277-3296  \
1     24인치 멀티확대 터치모니터  070-4481-4887   
2    24인치터치모니터 LED TV  070-4481-4887   
3       24인치화면확대터치모니터   02-2277-3296   
4        27인치 멀티터치모니터   02-2277

In [ ]:
#이미지
main_url='https://www.at4u.or.kr' #메인 링크
image_url_list=[] #이미지 링크 주소 리스트
num=[]

#count=0

for i in range(1,22): #페이지별 링크 주소 불러오기
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱

    url=soup.find_all('td',{'class':'align_center'})
    sub_url = [main_url + tag.find('img').get('src','') if tag.find('img') else '' for tag in url] #이미지 링크 불러오기
    #print(url)
    #print(sub_url)
    sub_url = list(filter(None,sub_url)) #이미지 링크 리스트에서 불필요한 공백 제거

    for i in range(len(sub_url)):
        image_url_list.append(sub_url[i]) #이미지 링크 리스트에 하나씩 추가
        #count+=1

for i in range(1,122): #제품번호 추가
    num.append(i)

#print(image_url_list)
df_image=pd.DataFrame(list(zip(num,image_url_list)),
                columns=['제품번호','제품이미지'])

print(df_image)

In [12]:
import re
import requests
from urllib.request import urlopen
import urllib.request 
from bs4 import BeautifulSoup 
from html.parser import HTMLParser



#이미지
main_url='https://www.at4u.or.kr' #메인 링크
image_url_list=[] #이미지 링크 주소 리스트
num=[]

#count=0

for i in range(1,22): #페이지별 링크 주소 불러오기
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱

    url=soup.find_all('td',{'class':'align_center'})
    sub_url = [main_url + tag.find('img').get('src','') if tag.find('img') else '' for tag in url] #이미지 링크 불러오기
    #print(url)
    #print(sub_url)
    sub_url = list(filter(None,sub_url)) #이미지 링크 리스트에서 불필요한 공백 제거

    for i in range(len(sub_url)):
        image_url_list.append(sub_url[i]) #이미지 링크 리스트에 하나씩 추가
        #count+=1

#print(image_url_list)
#print(image_url_list[0])

#res=urllib.request.urlopen(image_url_list[0])
#data=res.read()
#print(res)
#print(data)

#url=image_url_list[0]
#savename='test.png'
#mem=urllib.request.urlopen(url).read()

#with open(samename,mode='wb') as f:
#    f.write(mem)
#    print('저장')
    



In [42]:
import re
import requests
from urllib.request import urlopen
import urllib.request 
from bs4 import BeautifulSoup 
from html.parser import HTMLParser

In [39]:
url=image_url_list[0]
print(image_url_list[0])
path='C:/Users/User/Desktop/크롤링/이미지/{}.png'.format(0)

urllib.request.urlretrieve(main_url+url,path)
print('저장')

/FILES/PDT/040704140422%EB%A9%80%ED%8B%B0%EB%AA%A8%EB%8B%88%ED%84%B024_1.bmp
저장


In [41]:
#이미지 크롤링해서 폴더에 저장하기 최종

from urllib import parse

main_url='https://www.at4u.or.kr' #메인 링크
image_url_list=[] #이미지 링크 주소 리스트
#num=[]

#count=0

for i in range(1,22): #페이지별 링크 주소 불러오기
    response=requests.get('https://www.at4u.or.kr/F01000000000/F01030000000.asp?UpCode=&PDCode=&strPDTSupplyDate=2022&strGubun=&BoardConfig=&strSearch=&strKeyword=&intAbsolutePage={}'.format(i))
    soup=BeautifulSoup(response.content.decode('euc-kr','replace'),'html.parser') #html 문서 파싱

    url=soup.find_all('td',{'class':'align_center'})
    sub_url = [tag.find('img').get('src','') if tag.find('img') else '' for tag in url] #이미지 링크 불러오기
    #print(url)
    #print(sub_url)
    
    sub_url = list(filter(None,sub_url)) #이미지 링크 리스트에서 불필요한 공백 제거
    #print(sub_url)
    

    for i in range(len(sub_url)):
        encoding_url=urllib.parse.quote(sub_url[i]) # url 파싱
        image_url_list.append(encoding_url) #이미지 링크 리스트에 하나씩 추가
        #count+=1
        
for i in range(len(image_url_list)):
    url=image_url_list[i]
    path='C:/Users/User/Desktop/크롤링/이미지/{}.png'.format(i+1) #이미지 폴더에 제품 번호 값으로 이미지 저장
    urllib.request.urlretrieve(main_url+url,path) #이미지 파일 생성
        
#print(image_url_list)
#이미지 폴더에 파일 121개 생성됨!